In [1]:
import sys

sys.path.append("../")
import os
import math
import numpy as np
import cv2
import pandas as pd
import tikzplotlib
from pedrec.configs.dataset_configs import get_h36m_dataset_cfg_default
from pedrec.datasets.pedrec_dataset import PedRecDataset
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.visualizers.skeleton_3d_visualizer import add_skeleton_3d_to_axes
from pedrec.visualizers.visualization_helper_3d import draw_grid_3d, draw_origin_3d
from random import randint
import matplotlib.pyplot as plt
%matplotlib widget


In [2]:
cfg = PedRecNet50Config()

# ROM Train
dataset_cfg = get_h36m_dataset_cfg_default()
dataset_cfg.subsample = 1
dataset_root = "data/datasets/Human3.6m/train/"
dataset_df_filename = "h36m_train_pedrec.pkl"

dataset = PedRecDataset(dataset_root, dataset_df_filename, DatasetType.TRAIN, dataset_cfg, cfg.model.input_size, None)
dataset_length = len(dataset)
print(dataset_length)

1559752


In [4]:
fig, ax = plt.subplots(3,3, figsize=(10,10))
fig_3d = plt.figure(figsize=(10,10))

count = 0
for i in range(0, 3):
    for j in range(0, 3):
        entry = dataset[randint(0, dataset_length)]
        # entry = dataset[count + 606]
        model_input, labels = entry
        skeleton = labels["skeleton"]
        skeleton_3d = labels["skeleton_3d"]
        scale_factor = 3
        skeleton_3d[:, :3] *= scale_factor
        skeleton_3d[:, :3] -= (scale_factor / 2)
        center = labels["center"]
        scale = labels["scale"]
        rotation = labels["rotation"]
        is_real_img = labels["is_real_img"]
        img_path = labels["img_path"]
        skeleton[:, 0] *= model_input.shape[1]
        skeleton[:, 1] *= model_input.shape[0]
        visible_joints = np.sum(skeleton[:, 2])
        draw_skeleton(model_input, skeleton)
        img = model_input
        ax[i, j].imshow(img)
        ax[i, j].set_title(f"{count}: {visible_joints}")

        ax_3d = fig_3d.add_subplot(3, 3, count+1, projection='3d')
        draw_grid_3d(ax_3d, lim=1)
        draw_origin_3d(ax_3d)
        add_skeleton_3d_to_axes(ax_3d, skeleton_3d, size=4)
        ax_3d.set_title(f"{count}: {visible_joints}")

        # print(f"{count}: Model input shape: {model_input.shape}, Min value: {model_input.min()}, max value: {model_input.max()}")
        # print(f"{count}: center: {center}, scale: {scale}, rotation: {rotation}")
        # print(f"{count}: is_real_img: {is_real_img}")
        # print(f"{count}: visible joints: {visible_joints}")
        # print(f"{count}: path: {img_path}")
        # print("------------")
        count += 1
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
dataset_df_path = os.path.join(dataset_root, dataset_df_filename)
df = pd.read_pickle(dataset_df_path)
pd.options.display.max_columns = None
pd.options.display.float_format= '{:.2f}'.format
print(f"Number of entrys: {df.shape[0]}")

Number of entrys: 1559752


In [7]:
filter_skeleton2d = [col for col in df if col.startswith('skeleton2d')]
filter_skeleton3d = [col for col in df if col.startswith('skeleton3d')]
filter_bb = [col for col in df if col.startswith('bb')]
filter_body_orientation = [col for col in df if col.startswith('body_orientation')]
filter_head_orientation = [col for col in df if col.startswith('head_orientation')]
filter_env = [col for col in df if col.startswith('env')]

In [8]:
# skeleton 2d calculated columns
skeleton2d_xs = [col for col in df if col.startswith('skeleton2d') and col.endswith('_x')]
skeleton2d_ys = [col for col in df if col.startswith('skeleton2d') and col.endswith('_y')]
skeleton2d_visibles = [col for col in df if col.startswith('skeleton2d') and col.endswith('_visible')]
df["skeleton2d_width"] = df[skeleton2d_xs].max(axis=1) - df[skeleton2d_xs].min(axis=1)
df["skeleton2d_height"] = df[skeleton2d_ys].max(axis=1) - df[skeleton2d_ys].min(axis=1)
df["skeleton2d_size"] = np.sqrt(df["skeleton2d_width"]**2 + df["skeleton2d_height"]**2)
df["skeleton2d_visible_joints"] = df[skeleton2d_visibles].sum(axis=1)

In [9]:
# skeleton 3d calculated columns
skeleton3d_xs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_x')]
skeleton3d_ys = [col for col in df if col.startswith('skeleton3d') and col.endswith('_y')]
skeleton3d_zs = [col for col in df if col.startswith('skeleton3d') and col.endswith('_z')]
df["skeleton3d_width"] = df[skeleton3d_xs].max(axis=1) - df[skeleton3d_xs].min(axis=1)
df["skeleton3d_height"] = df[skeleton3d_ys].max(axis=1) - df[skeleton3d_ys].min(axis=1)
df["skeleton3d_depth"] = df[skeleton3d_zs].max(axis=1) - df[skeleton3d_zs].min(axis=1)
df["skeleton3d_size"] = np.sqrt(df["skeleton3d_width"]**2 + df["skeleton3d_height"]**2 + df["skeleton3d_depth"]**2)

In [10]:
# distance calculated column
env_position_xs = [col for col in df if col == "env_position_x"] 
env_position_ys = [col for col in df if col == "env_position_y"] 
env_position_zs = [col for col in df if col == "env_position_z"] 

df["distance_xz"] = np.sqrt(np.abs(df[env_position_xs].sum(axis=1)**2) + np.abs(df[env_position_zs].sum(axis=1)**2))

df["body_orientation_phi"] = df["body_orientation_phi"] * math.pi * 2
df["body_orientation_theta"] = df["body_orientation_theta"] * math.pi
df["head_orientation_phi"] = df["head_orientation_phi"] * math.pi * 2
df["head_orientation_theta"] = df["head_orientation_theta"] * math.pi


In [11]:
df.head(5)

,dataset,dataset_type,scene_id,scene_start,scene_end,frame_nr_global,frame_nr_local,img_dir,img_id,img_type,subject_id,gender,skin_color,size,bmi,age,movement,movement_speed,is_real_img,actions,bb_center_x,bb_center_y,bb_width,bb_height,bb_score,bb_class,env_position_x,env_position_y,env_position_z,body_orientation_theta,body_orientation_phi,body_orientation_score,body_orientation_visible,head_orientation_theta,head_orientation_phi,head_orientation_score,head_orientation_visible,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_nose_visible,skeleton2d_nose_supported,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_left_eye_visible,skeleton2d_left_eye_supported,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_right_eye_visible,skeleton2d_right_eye_supported,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_left_ear_visible,skeleton2d_left_ear_supported,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_right_ear_visible,skeleton2d_right_ear_supported,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_left_shoulder_visible,skeleton2d_left_shoulder_supported,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_right_shoulder_visible,skeleton2d_right_shoulder_supported,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_left_elbow_visible,skeleton2d_left_elbow_supported,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_right_elbow_visible,skeleton2d_right_elbow_supported,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_left_wrist_visible,skeleton2d_left_wrist_supported,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_right_wrist_visible,skeleton2d_right_wrist_supported,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_left_hip_visible,skeleton2d_left_hip_supported,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_right_hip_visible,skeleton2d_right_hip_supported,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_left_knee_visible,skeleton2d_left_knee_supported,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_right_knee_visible,skeleton2d_right_knee_supported,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_left_ankle_visible,skeleton2d_left_ankle_supported,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_right_ankle_visible,skeleton2d_right_ankle_supported,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_hip_center_visible,skeleton2d_hip_center_supported,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_spine_center_visible,skeleton2d_spine_center_supported,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_neck_visible,skeleton2d_neck_supported,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_lower_visible,skeleton2d_head_lower_supported,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_head_upper_visible,skeleton2d_head_upper_supported,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_left_foot_end_visible,skeleton2d_left_foot_end_supported,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_right_foot_end_visible,skeleton2d_right_foot_end_supported,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_left_hand_end_visible,skeleton2d_left_hand_end_supported,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d

In [12]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


,scene_start,scene_end,frame_nr_global,frame_nr_local,img_id,actions,bb_center_x,bb_center_y,bb_width,bb_height,bb_score,env_position_x,env_position_y,env_position_z,body_orientation_theta,body_orientation_phi,body_orientation_score,head_orientation_theta,head_orientation_phi,head_orientation_score,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d_right_hand_end_score,skeleton3d_nose_x,skeleton3d_nose_y,skeleton3d_nose_z,skeleton3d_nose_score,skeleton3d_left_eye_x,skeleton3d_left_eye_y,skeleton3d_left_eye_z,skeleton3d_left_eye_score,skeleton3d_right_eye_x,skeleton3d_right_eye_y,skeleton3d_right_eye_z,skeleton3d_right_eye_score,skeleton3d_left_ear_x,skeleton3d_left_ear_y,skeleton3d_left_ear_z,skeleton3d_left_ear_score,skeleton3d_right_ear_x,skeleton3d_right_ear_y,skeleton3d_right_ear_z,skeleton3d_right_ear_score,skeleton3d_left_shoulder_x,skeleton3d_left_shoulder_y,skeleton3d_left_shoulder_z,skeleton3d_left_shoulder_score,skeleton3d_right_shoulder_x,skeleton3d_right_shoulder_y,skeleton3d_right_shoulder_z,skeleton3d_right_shoulder_score,skeleton3d_left_elbow_x,skeleton3d_left_elbow_y,skeleton3d_left_elbow_z,skeleton3d_left_elbow_score,skeleton3d_right_elbow_x,skeleton3d_right_elbow_y,skeleton3d_right_elbow_z,skeleton3d_right_elbow_score,skeleton3d_left_wrist_x,skeleton3d_left_wrist_y,skeleton3d_left_wrist_z,skeleton3d_left_wrist_score,skeleton3d_right_wrist_x,skeleton3d_right_wrist_y,skeleton3d_right_wrist_z,skeleton3d_right_wrist_score,skeleton3d_left_hip_x,skeleton3d_left_hip_y,skeleton3d_left_hip_z,skeleton3d_left_hip_score,skeleton3d_right_hip_x,skeleton3d_right_hip_y,skeleton3d_right_hip_z,skeleton3d_right_hip_score,skeleton3d_left_knee_x,skeleton3d_left_knee_y,skeleton3d_left_knee_z,skeleton3d_left_knee_score,skeleton3d_right_knee_x,skeleton3d_right_knee_y,skeleton3d_right_knee_z,skeleton3d_right_knee_score,skeleton3d_left_ankle_x,skeleton3d_left_ankle_y,skeleton3d_left_ankle_z,skeleton3d_left_ankle_score,skeleton3d_right_ankle_x,skeleton3d_right_ankle_y,skeleton3d_right_ankle_z,skeleton3d_right_ankle_score,skeleton3d_hip_center_x,skeleton3d_h

In [13]:
df_description = df.describe()
num_3d_models = 1
num_animations = 1
print("General Report")
print(f"Number of frames: {len(df)}")
print(f"FPS: 30")
print(f"Resolution: 1920x1080")
print(f"Number of 3D models: {num_3d_models}")
print(f"Number of animations {num_animations}")
print("Data & Mean & Std & Min & Max \\\\")
print(f"Skeleton 2D Diameter & ${df_description['skeleton2d_size']['mean']:.2f}px$ & ${df_description['skeleton2d_size']['std']:.2f}px$ & ${df_description['skeleton2d_size']['min']:.2f}px$ & ${df_description['skeleton2d_size']['max']:.2f}px$ \\\\")
print(f"Skeleton 3D Diameter & ${df_description['skeleton3d_size']['mean']:.2f}mm$ & ${df_description['skeleton3d_size']['std']:.2f}mm$ & ${df_description['skeleton3d_size']['min']:.2f}mm$ & ${df_description['skeleton3d_size']['max']:.2f}mm$ \\\\")
print(f"Camera Distance (XZ) & ${df_description['distance_xz']['mean']:.2f}mm$ & ${df_description['distance_xz']['std']:.2f}mm$ & ${df_description['distance_xz']['min']:.2f}mm$ & ${df_description['distance_xz']['max']:.2f}mm$ \\\\")

General Report
Number of frames: 1559752
FPS: 30
Resolution: 1920x1080
Number of 3D models: 1
Number of animations 1
Data & Mean & Std & Min & Max \\
Skeleton 2D Diameter & $855.46px$ & $83.12px$ & $585.43px$ & $1339.59px$ \\
Skeleton 3D Diameter & $1712.69mm$ & $197.96mm$ & $859.04mm$ & $2769.01mm$ \\
Camera Distance (XZ) & $5170.68mm$ & $754.95mm$ & $2532.69mm$ & $7690.65mm$ \\


In [ ]:
fig = plt.figure()
df['skeleton2d_width'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton2d_height'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton2d_size'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton3d_width'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton3d_height'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton3d_depth'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton3d_size'].hist(bins=50)

In [2]:
from torchvision import transforms
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
inv_trans = transforms.Compose([transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                     std = [ 1., 1., 1. ]),
                               ])
cfg = PedRecNet50Config()

# ROM Train
dataset_cfg = get_h36m_dataset_cfg_default()
dataset_cfg.subsample = 1
dataset_root = "data/datasets/Human3.6m/train/"
dataset_df_filename = "h36m_train_pedrec.pkl"

dataset = PedRecDataset(dataset_root, dataset_df_filename, DatasetType.VALIDATE, dataset_cfg, cfg.model.input_size, trans)
dataset_length = len(dataset)
print(dataset_length)

1559752


In [3]:
model_input, labels = dataset[11897]
img_gt = inv_trans(model_input).detach().cpu().numpy()
img_gt *= 255
img_gt = np.transpose(img_gt, (1, 2, 0))
img_gt = img_gt.astype(np.uint8)
# img_gt = inv_trans(model_input).detach().cpu().numpy()
# img_gt *= 255
# img_gt = np.transpose(img_gt, (1, 2, 0))
# img_gt = img_gt.astype(np.uint8)
# Hack because cv2 bugs on draw_circle etc. without this
# img_gt = cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB)
fig = plt.figure()
fig.suptitle(f"{11897}")
plt.imshow(img_gt)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
from pedrec.networks.net_pedrec.pedrec_net import PedRecNet, PedRecNetLossHead
from pedrec.networks.net_pedrec.pedrec_net_mtl_wrapper import PedRecNetMTLWrapper
from pedrec.training.experiments.experiment_path_helper import get_experiment_paths_home
import torch
from pedrec.utils.torch_utils.torch_helper import get_device, move_to_device

experiment_paths = get_experiment_paths_home()
device = get_device()
net = PedRecNet(cfg)
loss_head = PedRecNetLossHead(device)
net.init_weights()
# initialize_pose_resnet(net, experiment_paths.pose_resnet_weights_path)  # load COCO pretrained weights
# initialize_from_coco_trained_v2(net, experiment_paths.pose_2d_coco_only_weights_path)
net = PedRecNetMTLWrapper(net, loss_head)
net.load_state_dict(torch.load("data/models/pedrec/experiment_pedrec_p2d3d_c_o_h36m_sim_mebow_0.pth"))
net.to(device)
net.eval()
print("network loaded!")

network loaded!


In [5]:
model_input = model_input.unsqueeze(0)
model_input = model_input.to(device)
labels["skeleton"] = torch.from_numpy(labels["skeleton"]).unsqueeze(0)
labels["skeleton_3d"] = torch.from_numpy(labels["skeleton_3d"]).unsqueeze(0)
labels["orientation"] = torch.from_numpy(labels["orientation"]).unsqueeze(0)
labels = move_to_device(labels, device)
outputs, loss = net(model_input, labels)

/usr/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /build/python-pytorch/src/pytorch-1.9.0-opt-cuda/c10/core/TensorImpl.h:1153.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [6]:
from pedrec.models.constants.skeleton_pedrec import SKELETON_PEDREC_JOINTS
from scipy.special import softmax
pose_2d_map = outputs[3].detach().cpu().numpy()
# depth_map = outputs[2].detach().cpu().numpy()
# pose_2d_map_softmax = softmax(pose_2d_map[0, :, :, :])
pose_2d_map_softmax = softmax(pose_2d_map[0, :, :, :])
test = np.amax(pose_2d_map_softmax[2])
print(pose_2d_map_softmax.shape)
print(test)

joint_num = 13
fig = plt.figure()
# fig.suptitle(f"Heatmap ({SKELETON_PEDREC_JOINTS[joint_num].name})")
plt.imshow(pose_2d_map_softmax[joint_num], cmap='hot', interpolation='nearest')
plt.show()
# plt.imsave("/home/dennis/code/latex/Dissertation/gfx/pedrec_example_maps/p2d_softmax_lknee_bceloss.png", pose_2d_map[joint_num], cmap='hot')

(26, 64, 48)
0.000119160075


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
pose_3d_map = outputs[4].detach().cpu().numpy()
# depth_map = outputs[2].detach().cpu().numpy()
# pose_2d_map_softmax = softmax(pose_2d_map[0, :, :, :])
pose_3d_map_softmax = softmax(pose_3d_map[0, :, :, :])
test = np.amax(pose_3d_map_softmax)
print(pose_3d_map_softmax.shape)
print(test)
# 316 408 157 48
joint_num = 13
fig = plt.figure()
# fig.suptitle(f"Heatmap ({SKELETON_PEDREC_JOINTS[joint_num].name})")
plt.imshow(pose_3d_map_softmax[joint_num], cmap='hot', interpolation='nearest')
plt.show()
# plt.imsave("/home/dennis/code/latex/Dissertation/gfx/pedrec_example_maps/p2d_softmax_lknee_bceloss.png", pose_2d_map[joint_num], cmap='hot')

(26, 64, 48)
0.0015037217


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
output_orientations = outputs[2].detach().cpu().numpy()
theta_maps = outputs[5].detach().cpu().numpy()
phi_maps = outputs[6].detach().cpu().numpy()
# pose_3d_map_softmax = softmax(pose_3d_map[0:, :, :])
## Draw heatmaps for body orientations
fig = plt.figure(figsize = (10, 4))
plt.xticks(fontsize=20)
plt.yticks([])
body_theta_net = output_orientations[0, 0, 0]
body_theta = math.degrees(body_theta_net * math.pi)
print(f"Body Orientation (vertical) (θ_net={body_theta_net:.2f} / θ = {int(body_theta)}°)")
fig.suptitle(f"Body Orientation (vertical) (θ_net={body_theta_net:.2f} / θ = {int(body_theta)}°)", fontsize=20, y=1.3)
plt.imshow(np.expand_dims(softmax(theta_maps[0, 0, :]), axis=0), cmap='hot', interpolation='nearest', aspect='auto')
plt.show()

fig = plt.figure(figsize = (10, 4))
plt.xticks(fontsize=20)
plt.yticks([])
body_phi_net = output_orientations[0, 0, 1]
body_phi = math.degrees(body_phi_net * 2 * math.pi)
print(f"Body Orientation (horizontal) (φ_net = {body_phi_net:.2f} / φ = {int(body_phi)}°)")
fig.suptitle(f"Body Orientation (horizontal) (φ_net = {body_phi_net:.2f} / φ = {int(body_phi)}°)", fontsize=20, y=1.3)
plt.imshow(np.expand_dims(softmax(phi_maps[0, 0, :]), axis=0), cmap='hot', interpolation='nearest', aspect='auto')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Body Orientation (vertical) (θ_net=0.50 / θ = 90°)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Body Orientation (horizontal) (φ_net = 0.85 / φ = 305°)


In [21]:
output_orientations = outputs[2].detach().cpu().numpy()
theta_maps = outputs[5].detach().cpu().numpy()
phi_maps = outputs[6].detach().cpu().numpy()
# pose_3d_map_softmax = softmax(pose_3d_map[0:, :, :])
## Draw heatmaps for head orientations
fig = plt.figure(figsize = (10, 4))
plt.xticks(fontsize=20)
plt.yticks([])
head_theta_net = output_orientations[0, 1, 0]
head_theta = math.degrees(head_theta_net * math.pi)
print(f"head Orientation (vertical) (θ_net={head_theta_net:.2f} / θ = {int(head_theta)}°)")
fig.suptitle(f"head Orientation (vertical) (θ_net={head_theta_net:.2f} / θ = {int(head_theta)}°)", fontsize=20, y=1.3)
plt.imshow(np.expand_dims(softmax(theta_maps[0, 1, :]), axis=0), cmap='hot', interpolation='nearest', aspect='auto')
plt.show()

fig = plt.figure(figsize = (10, 4))
plt.xticks(fontsize=20)
plt.yticks([])
head_phi_net = output_orientations[0, 1, 1]
head_phi = math.degrees(head_phi_net * 2 * math.pi)
print(f"head Orientation (horizontal) (φ_net = {head_phi_net:.2f} / φ = {int(head_phi)}°)")
fig.suptitle(f"head Orientation (horizontal) (φ_net = {head_phi_net:.2f} / φ = {int(head_phi)}°)", fontsize=20, y=1.3)
plt.imshow(np.expand_dims(softmax(phi_maps[0, 1, :]), axis=0), cmap='hot', interpolation='nearest', aspect='auto')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

head Orientation (vertical) (θ_net=0.49 / θ = 88°)


<ipython-input-21-0a7b5667cd75>:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize = (10, 2))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

head Orientation (horizontal) (φ_net = 0.98 / φ = 352°)
